In [16]:
import time
import math
import threading
import cv2
import numpy as np
from dronekit import connect, VehicleMode, LocationGlobalRelative, APIException
from flask import Flask, render_template, jsonify, request, Response
from flask_socketio import SocketIO, emit
from pymavlink import mavutil


In [17]:
# --- Global Variables ---
vehicle = None
telemetry_data = {}
update_interval = 1.0
running = True  # Flag to control background threads

# --- Computer Vision Variables ---
cap = None  # Webcam capture object
frame = None  # Current frame
mango_detected = False
mango_position = (0, 0)  # (x, y) center of detected mango
frame_center = (0, 0)  # Will be calculated when webcam starts
mango_tracking_active = False
cv_lock = threading.Lock()  

## Connecting a vehicle

In [18]:
# Connect to vehicle
print("Connecting to vehicle...")
vehicle = connect('tcp:127.0.0.1:5762', wait_ready=True)
print("Connected.")

Connecting to vehicle...
Connected.


CRITICAL:autopilot:Arm: Need Position Estimate
CRITICAL:autopilot:PreArm: Need Position Estimate


In [11]:
pip install torch torchvision torchaudio ultralytics

  Using cached torchvision-0.22.0-cp39-cp39-win_amd64.whl (1.7 MB)
  Using cached ultralytics-8.3.127-py3-none-any.whl (1.0 MB)
  Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
  Using cached fsspec-2025.3.2-py3-none-any.whl (194 kB)
  Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
  Using cached pillow-11.2.1-cp39-cp39-win_amd64.whl (2.7 MB)
  Using cached matplotlib-3.9.4-cp39-cp39-win_amd64.whl (7.8 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl (162 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl (11.6 MB)
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB

You should consider upgrading via the 'c:\Users\saira\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [19]:

try:
    # For YOLOv8, we need to use ultralytics package instead of torch.hub
    from ultralytics import YOLO
    
    # Load the YOLOv8 model
    model = YOLO('predict_mango.pt')  # Load a custom model
    
    print("YOLOv8 model loaded successfully.")

except ImportError:
    print("Error: PyTorch not found. Please install it (pip install torch torchvision torchaudio).")
    model = None
except Exception as e:
    print(f"Error loading YOLO model: {e}")
    model = None

YOLOv8 model loaded successfully.


In [ ]:
def initialize_webcam():
    global cap, frame_center
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        height, width = frame.shape[:2]
        frame_center = (width // 2, height // 2)
        results = model(frame)
        annotated_frame = results[0].plot()
        cv2.imshow("Objec detection model", annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
        # cap = cv2.VideoCapture(0)  # Use 0 for default webcam
        # if not cap.isOpened():
        #     print("Error: Could not open webcam.")
        #     return False
        
        # # Get webcam frame dimensions
        # ret, test_frame = cap.read()
        # if ret:
        #     height, width = test_frame.shape[:2]
        #     frame_center = (width // 2, height // 2)
        #     print(f"Webcam initialized. Frame size: {width}x{height}, Center: {frame_center}")
        #     return True
        # else:
        #     print("Error: Failed to read frame from webcam.")
        #     return False
    
    
def detect_mango(image):
    """
    Detect mangoes in the image using the YOLOv8 model.
    Returns (success, center_x, center_y, width, height) of the first detected mango.
    """
    global model, frame_center
    if model is None:
        print("YOLO model not loaded.")
        return False, 0, 0, 0, 0

    try:
        # Perform inference with YOLOv8
        results = model(image, verbose=False)  # Suppress extra output
        
        # YOLOv8 returns a list of Results objects - get the first one
        result = results[0] if results else None
        
        if result is None:
            return False, 0, 0, 0, 0
        
        # Get all detections
        boxes = result.boxes
        
        # Check if any detections exist
        if len(boxes) == 0:
            return False, 0, 0, 0, 0
            
        # Find mango detections - YOLOv8 uses names in model.names
        mango_detections = []
        for i, box in enumerate(boxes):
            # Get class id
            cls_id = int(box.cls.item()) if hasattr(box, 'cls') else -1
            # Check if this class is a mango
            if cls_id >= 0 and cls_id < len(model.names) and 'mango' in model.names[cls_id].lower():
                mango_detections.append(box)
            
        # If no mangoes found
        if not mango_detections:
            return False, 0, 0, 0, 0
            
        # Get first mango detection
        mango_box = mango_detections[0]
        
        # Extract bounding box
        x1, y1, x2, y2 = map(int, mango_box.xyxy[0].tolist())
        
        # Calculate center and dimensions
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        width = x2 - x1
        height = y2 - y1
        
        return True, center_x, center_y, width, height
    except:
        pass


In [27]:
initialize_webcam()

Webcam initialized. Frame size: 640x480, Center: (320, 240)


True

In [20]:
def arm_and_takeoff(target_altitude):
    print("Arming...")
    vehicle.mode = VehicleMode("GUIDED")
    vehicle.armed = True
    while not vehicle.armed:
        print(" Waiting for arming...")
        time.sleep(1)
    print("Taking off...")
    vehicle.simple_takeoff(target_altitude)
    while True:
        alt = vehicle.location.global_relative_frame.alt
        print(f" Altitude: {alt:.2f} m")
        if alt >= target_altitude * 0.95:
            print("Reached target altitude.")
            break
        time.sleep(1)

def condition_yaw(heading, relative=True, yaw_speed=20):
    is_relative = 1 if relative else 0
    msg = vehicle.message_factory.command_long_encode(
        0, 0,
        mavutil.mavlink.MAV_CMD_CONDITION_YAW,
        0,
        heading, yaw_speed, 1, is_relative,
        0, 0, 0
    )
    vehicle.send_mavlink(msg)
    vehicle.flush()